# Welcome to ASIC 2024

We are now at the live demo portion of the tutorial. You can either clone (or download) this notebook file from the tutorial GitHub repository or follow along in your own environment. At this point, we have already walked through how to install `atmospy` and its dependencies.

**Note: no need to take notes on this. This entire notebook has been uploaded to the ASIC-2024 GitHub repository.**

In [ ]:
import atmospy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

# disable warnings for demo purposes
warnings.filterwarnings("ignore")

atmospy.set_theme()

# this tutorial was completed using version:
atmospy.__version__

# Data

When working with air sensor data, you will almost always want to be working with data in the form of a DataFrame. Likely, this means you will be using the `pandas` library to do so. A DataFrame holds 2-dimensional data, consisting of columns and rows, much like a spreadsheet. `atmospy` assumes all data is a DataFrame and doesn't currently support anything else.

## Formatting your data to be atmospy compliant

While I tried not to make too many assumptions about data types or structure, in order to make the librar functional, `atmospy` makes a couple of key assumptions about data structure which you must follow in order to use the library as intended:

  1. Your data must be a `DataFrame`
  2. If you are making a figure that requires a timestamp, your column containing the timestamp must be a proper datetime object (not a string)
  3. Any columns containing your 'values' must be of numeric type


If you aren't familiar with `DataFrames`, I highly recommend getting your hands on Wes McKinney's book on **Python for Data Analysis**. Below, we'll cover a few tricks for ensuring your data is in proper format in the event you aren't familiar.

## Dealing with timestamps

If you've ever worked with data, you will know there are *a ton* of different timestamp formats you may encounter. Pandas (and python generally) have some useful tools for converting a string-formatted timestamp (e.g., "2024-01-01 01:23:45") to a proper python datetime. 

In [ ]:
tstamp_as_string = "2024-01-01 01:23:45"

type(tstamp_as_string)

Let's convert this to a datetime object using the `to_datetime` function in `pandas`:

In [ ]:
tstamp = pd.to_datetime(tstamp_as_string)

type(tstamp)

Easy enough! Now, how do you know what type your data is? You can inspect the DataFrame quite easily using the `info()` method on a DataFrame:

In [ ]:
df = pd.DataFrame(
    columns=['timestamp', 'A', 'B'], 
    data=[["2024-01-01", 1, 2], ["2024-01-02", "2", 3]]
)

df.info()

We just created a DataFrame (not a very interesting one, but we're just trying to showcase something). By inspecting the DataFrame, we can see that we have one column - the column that contains a timestamp - that is an object while the other two are integers. Using the `to_datetime` method we looked at above, let's convert our `timestamp` column to a real live timestamp:

In [ ]:
df["timestamp"] = df["timestamp"].map(pd.to_datetime)

df.info()

As you can see by looking at the data type, we now have a true datetime object!

## Dealing with non-numeric types

Now, what about the two data columns ("A" and "B"). While column "B" is already numeric (it's an integer), column "A" isn't! Why not? Well, if you noticed above when we created this (very fake) DataFrame, I explicitly forced one of the numbers to be a string by including quotes around it. While you may think this is rare and/or not something you will encounter in the real world, I can absolutely assure you that you are wrong. It is *very* common that when working with reference data, otherwise numeric columns will include symbols or other non-numeric characters to signify calibration periods or other downtime. 

While the manufacturer's of these instruments may be breaking every data recording law on the books (not a real thing), it very much happens and can be frustrating to deal with. Fortunately, `pandas` has a fairly robust way of dealing with it. You can force a column to be numeric and tell it what you want to do with values that it can't force to a numeric type:

In [ ]:
df["A"] = pd.to_numeric(df["A"], errors='coerce')

df.info()

We used the `to_numeric` function available in `pandas` and told it to *coerce* any values that were invalid to be NaN. While this didn't actually happen in this very small dataset, it can be useful in real-life scenarios. 

## How do I load my data?

So, how do you actually load a csv (or other) file into Python and get it to be a DataFrame? Well, there are quite a few ways; however, i'm going to show you the easiest one that uses only dependencies you already have installed by function of this library.

The `pandas` library has incredibly robust utility functions for loading data from csv, text file, excel, or more. At it's simplist, you can run the single line below and load your file directly into a pandas dataframe. 

In [ ]:
# df = pd.read_csv("<path-to-file-here>")

We're not going to spend too much time on this during this tutorial, so we are going to move on to discussing DataFrame structure, but if you have questions or need additional resources, my two favorite libraries for loading data are `pandas` and `dask`, which is especially useful if working with large datasets.

## Long-form vs wide-Form data

**Note: In order to explain what I mean in this next section, I am going to load an example dataset, even though we won't learn how to do that for a few more minutes.**

There are multiple ways in which you can organize a DataFrame (or spreadsheet or database) and the best way really depends on what you're trying to do. There are two common ways to organize a DataFrame:

1. **Wide Form**. Where columns and rows contain levels of different variables, much like reading a spreadsheet.
1. **Long Form**. Where each variable gets its own column and each observation is a row.


For a much more throughout explanation of wide vs. long form data, check out Michael Waskom's description [here](https://seaborn.pydata.org/tutorial/data_structure.html#long-form-vs-wide-form-data) or Hadley Wickham's paper [here](https://vita.had.co.nz/papers/tidy-data.pdf).

Rather than try to continue explaining this in words, I will just show you. To do this, I will enlist the help of one of `atmospy`'s example datasets.

In [ ]:
# this block is a last resort in the event internet is not functioning

# bc = pd.read_csv(atmospy.utils.get_data_home() + "/us-bc.csv")
# bc["Timestamp GMT"] = bc["Timestamp GMT"].map(pd.to_datetime)
# bc["Timestamp Local"] = bc.apply(
#             lambda x: x["Timestamp GMT"] + pd.Timedelta(hours=x["GMT Offset"]), axis=1)
# bc.head()

In [ ]:
df = atmospy.load_dataset("air-sensors-pm")

# keep only the first few records
df = df.head(15)

df

So, we just loaded an example dataset that contains co-located air sensors (Sensor A, Sensor B, and Sensor C) with an EPA FEM monitor (Reference). By inspecting the data, we can see that we have hourly resolution data and for now, I'm only looking at the first 15h of data. Depending on how you do your analysis, this is a very real spreadsheet that you may have and can read into Python. This is **wide-form** data. For each timestamp, we have four observations; however, they're all in the same row! This can be super useful if we wanted to explore the pairwise relationship between all sensors AND reference monitor. But, what if we wanted to easily plot each sensor against the reference, but not each other?

We can convert this to **long-form** data by using `pandas`' `melt` function:

In [ ]:
df.melt(id_vars=["timestamp", "Reference"], value_name="Sensor")

While I didn't choose particularly useful column names, you can see that we now have a long-form dataframe where each observation includes both the Reference value ("Reference") AND the Sensor value ("Sensor") with the identifier for the sensor in the "variable" column.

I know this is a very short (hopefully not useless) primer on data formatting and types, it will be highly relevant when it comes to actually plotting your data, which we're about to (finally) get to. There are a ton of great resources available for learning more about this subject, two of which are already listed above.

### Example Datasets

In order to actually make plots, we need data. While you can easily make synthetic datasets, we've included a few real datasets that are available for you to use. 

| Name | Description |
|:----:|:------------|
| `us-ozone` | Ozone data for 2023 from a subset of all US reference sites, directly from EPA |
| `us-bc` | Black Carbon data for 2023 from a subset of all US reference sites, directly from EPA |
|`air-sensors-pm`| Co-located data from three PM sensors and a FEM reference monitor for a period of ~8 weeks |
|`air-sensors-met`| Data from a QuantAQ MODULAIR at a random location with wind direction and wind speed data |


It is likely the available datasets will change over time and there is no guarantee these exact ones will remain. You can always check which ones are available by using the utility function `get_dataset_names`:

In [ ]:
atmospy.get_dataset_names()

When you run this for the first time, it will cache it locally on your system so that subsequent calls are much quicker. To load a dataset, you can call `load_dataset`:

In [ ]:
ozone = atmospy.load_dataset("us-ozone")

ozone.info()

There is the `us-ozone` dataset that includes ~100 unique locations from across the US from 2023. This data was gathered directly from EPA's website and is all federal reference data.

In [ ]:
bc = atmospy.load_dataset("us-bc")
bc.info()

Next is Black Carbon. Just like ozone, this is data from the EPA for 2023 and was downloaded directly from EPA's website.

In [ ]:
pm = atmospy.load_dataset("air-sensors-pm")
pm

Next up is a dataset that includes co-located sensor data with a reference monitor. These are all PM2.5 measurements, FYI.

In [ ]:
met = atmospy.load_dataset("air-sensors-met")
met.info()

Last is another air sensor dataset. This one is a random sensor that just so happens to have wind speed and wind direction with it which makes it useful for some of the figures we'll make. Otherwise, it is an entirely uninteresting dataset.


If you have more interesting datasets that you're willing to allow others to use, please feel free to reach out to me and I can add them to the `atmospy-data` repository if that's of mutual interest.

# Atmospy

## Where does it fit?

Like `seaborn`, `atmospy` is a wrapper around `matplotlib` (as well as `seaborn` itself). It is designed to be a tool that you *can* use and it is designed to make common figures that are useful in air sensor work a little bit easier to make. Because it is simply a `matplotlib` wrapper, you can modify the axis object that each function returns and continue to customize these figures in near limitless ways.


We are going to walk through each of the fundamental figures that exist in `atmospy` today and talk about the type of story you can tell with each one as well as some of the nuts and bolts of actually using the function and it's abilities. 


 <!-- 1. Go over each figure with defaults to show them off
 2. Overview how and why you may need to edit them and/or choose different values
 3. Talk about matplotlib and seaborn and where to get more info -->

## Visualizing Linear Relationships

When working with air sensor data, it is quite often that want, or need, to compare two sensors or one air sensor to a reference monitor. Often, this takes the form of a scatter plot between the two. This is what we refer to as a regression plot. In `atmospy`, we call it a `regplot` and it's as easy as providing the function your input dataframe and defining the x and y columns to use as your variables:

In [ ]:
g = atmospy.regplot(
    data=pm,
    x="Reference", 
    y="Sensor A"
)

This may - or may not - look like what you expected. What is the story we're trying to tell with a simple regression plot? At least in the air sensor context, we are often trying to describe how well two different instruments agree with one another. Often, we use a lienar model to describe this and report the fit of them model to describe the relationship.

In the default `regplot` figure, you see the following:
  * each pair of observations is shown as a single scatter point. They have white edges so that you can (somewhat) tell the difference between different observations.
  * We add a best-fit line (OLS) and label the line with the fit parameters in the legend
  * The aspect ratio of the figure is 1 so that you can easily tell if the slope were to be very high or very low
  * We don't color by a separate variable (like temperature or humidity) - that's not the story we're trying to tell here and it clutters the figure
  * we plot the distribution of each variable on the joint axes. This allows you to easily see what the underlying distribution of data was, which can provide useful context for a fit model
  * we don't have too many axis ticks or labels - they're not necessary here. 


What do you think? What would you do differently?

Personally, I don't think there is too much I would change about this figure in order to help it tell the story and convey the information more clearly. However, there are a number of ways you can configure the plot based on your needs.

For example, you can pass along keyword arguments to control the look of the joint axes. In this case, maybe we want to add a kernel density estimate to the distributions:

In [ ]:
g = atmospy.regplot(
    data=pm,
    x="Reference", 
    y="Sensor A",
    marginal_kws={
        "kde": True
    }
)

Or, maybe we want to change the marker style and color:

In [ ]:
g = atmospy.regplot(
    data=pm,
    x="Reference", 
    y="Sensor A",
    color="g",
    marker='^'
)

Or, maybe we don't actually *want* to add the best-fit line:

In [ ]:
g = atmospy.regplot(
    data=pm,
    x="Reference", 
    y="Sensor A",
    fit_reg=False
)

If there is something that isn't possible to modify using the typical arguments of the function, no worries! We can also modify after the fact. Say we want to change the x-axis label to be more explicit:

In [ ]:
g = atmospy.regplot(
    data=pm,
    x="Reference", 
    y="Sensor A",
    fit_reg=False
)
g.set_axis_labels(xlabel="Teledyne T640 $PM_{2.5}$ [$µgm^{-3}$]");

The return from the `atmospy` function is just a `matplotlib` Axes object or `seaborn` JointGrid object, so modifying them is quite easy.

## Visualizing Diel Trends

A diel cycle is a pattern that recurs every 24h. We often see diel trends with pollutants that are driven by photochemistry such as ozone, which typically peaks in the early-to-mid afternoon each day and then decreases. We can also find diel patterns associated with human activity such as traffic patterns.

To make a diel plot, we typically have to do a good amount of data munging first including determining the mean value at a given time of day, every day, and then grouping those together into a single figure.


...or you can use `atmospy.dielplot`:

In [ ]:
# let's limit our data to a single site 
ozone_single_site = ozone[
    ozone["Local Site Name"] == ozone["Local Site Name"].unique()[0]
]

ax = atmospy.dielplot(
    data=ozone_single_site,
    x="Timestamp Local",
    y="Sample Measurement",
)

Above, we show the figure with the default settings and, well, it leaves a bit to be desired, though it does have the basics. We see the diel trend shown in the darker blue line and the IQR shown in the shaded blue line. Personally, I would like to see the y-axis go to zero, i'd like to see some labels, and i'd like to see the main diel line be a bit darker and more pronounced.

Let's go ahead and quickly make those changes:

In [ ]:
ax = atmospy.dielplot(
    data=ozone_single_site,
    x="Timestamp Local",
    y="Sample Measurement",
    ylabel="$O_3$ [ppbv]",
    ylim=(0, None),
    plot_kws={
        "lw": 5
    }   
)

That's a bit better, though could still be improved a bit. Overall, it tells us what we're looking for. On a repeating, 24h cycle, ozone tends to be lowest around 5 or 6 AM and highest around 3 PM local time. It's not overly complicated and isn't too busy. 

What information is this figure still missing that might be useful to convey? Maybe some more information about the distribution? Do we really need this to be a continuous line? Or could it be some sort of bar chart that is grouped by hour? 

What about the information conveyed? How else could we break up, or subselect the data, to be more specific? What if we were plotting CO next to a roadway rather than ozone, which is going to be more regionally consistent?

Anyways, more on how we can explore and graph some of those things in a bit.

While the diel plot shows you the average (with some statistics) of the 24h cycle, sometimes, we want to look at how the deil cycle changes on a day-to-day basis. 

## Visualizing Short-to-Medium Term Trends

We can use a variant of a calendar plot to easily graph this, where we plot the hour of day on one axis and the day of week or day of month on the other axis. In `atmospy`, you can use the `calenderplot` function to do this:

In [ ]:
ax = atmospy.calendarplot(
    data=ozone_single_site,
    x="Timestamp Local",
    y="Sample Measurement",
    freq="hour",
    xlabel="Day of Month",
    ylabel="Time of Day",
    height=4,
    vmin=0, vmax=80,
    cbar=False,
    cmap="flare"
)

On the y-axis here, we go from midnight to midnight top to bottom and on the x-axis we go from the first day of the month to the last. This is a quick and easy way to view quite a bit of data at once - we're showing the mean hourly ozone values in a way that allows your eye to detect subtle trends:

  * we have consistent peaks between 1 PM and 6 PM
  * we have a couple of days where reported ozone is fairly high even at easlry hours (~4 days in this month?)
  * we're missing almost 2 days of data!


It would be quite hard to convey this much information with a timeseries, do you agree?

By default, we plot the mean value in a given hour. However, you can configure it to plot by any other aggregation as well if you so choose (e.g., max, min, median). Simply change the `agg` argument and you're good to go. We'll show here for completeness, though with ozone, we don't expect much of a difference between mean and max.

In [ ]:
ax = atmospy.calendarplot(
    data=ozone_single_site,
    x="Timestamp Local",
    y="Sample Measurement",
    freq="hour",
    xlabel="Day of Month",
    ylabel="Time of Day",
    height=4,
    vmin=0, vmax=80,
    cbar=False,
    cmap="flare",
    agg="max"
)

I would argue there are better ways to show this in a cleaner and more concise manner, but this is a figure I get asked about a lot.

## Visualizing Long-Term Trends

While you could quite easily expand the x-axis above and include all 365 days in a year, do you really need to show the diel trend for an entire year at once? Almost certaintly not...


However, it can be very useful to show long-term trends in a pollutant. While there are several good ways to do this, one approach would be to use a `calendarplot` like above, but rather than showing hourly values, we show daily values:

In [ ]:
atmospy.calendarplot(
    data=ozone_single_site,
    x="Timestamp Local",
    y="Sample Measurement",
    freq="day",
    cbar=False,
    height=2,
    linewidths=0.1,
);

The lack of data completeness aside from the random ozone monitor we chose aside...


In this figure, each column is a week and each row corresponds to a day of the week. Here' we're visualizing the mean ozone value by day over the course of a (albeit stunted) year. 

What story can you tell with a figure like this? Well, for one, I would bet that if you plotted this for PM2.5 somewhere on the East Coast during 2023, you would mostly see values lower than 15 µg/m3 with the key exception of a couple of weeks where PM2.5 was incredibly high due to wildfires.

Next, we're going to move onto the last of our core four figures in `atmospy` before moving on to some more advanced techniques.

## Visualing Pollutant Source

Identifying the source of pollution is one of the most common questions we ask. This can be an incredibly challenging task and is tackled in a number of different ways. One way that we can begin to tell a story is by looking at from which direction a pollutant is orgininating. A wind rose is a common figure for depicting the direction and strength of wind. Here, we show a variant of the wind rose called a pollution rose, which is quite similar, though instead of visualizing the wind speed, we're visualizing the pollution intensity:

In [ ]:
# load an example dataset with MET info
met = atmospy.load_dataset("air-sensors-met")

atmospy.pollutionroseplot(
    data=met,
    ws="ws",
    wd="wd",
    pollutant="pm25",
    calm=0.0,
);

Above, we used the `pollutionroseplot` available in atmospy to show the directionality and intensity of PM2.5.

So what's going on in this figure? We've taken the combination of wind speed, wind direction, and PM2.5 data and binned it both along the theta using wind direction as well as the radius using the PM2.5 value. Both variables are grouped/binned and those values are configurable by you using the `segments` (wind direction) and `bins` (pollutant) arguments.

So where does wind speed come in to play? Our goal is to show the direction from where the PM2.5 came. If we are under calm wind conditions, we may not have reason to believe that the wind direction we record is actually true or valid. Therefore, we don't use any of the data that we consider to be under "calm" wind conditions. Again, this is configurable by the user using the `calm` argument. We visualize this by leaving the center of the figure blank with an area that is proportional to the percentage of calm winds. To quickly visualize this, let's first set `calm=-10` and then `calm=10`.

In [ ]:
atmospy.pollutionroseplot(
    data=met,
    ws="ws",
    wd="wd",
    pollutant="pm25",
    calm=-10.0,
);

In [ ]:
atmospy.pollutionroseplot(
    data=met,
    ws="ws",
    wd="wd",
    pollutant="pm25",
    calm=10.0,
);

It becomes quite clear that this center bit has increased dramatically to convey that a larger percentage of our data occured under 'calm' wind conditions.

So how do interpret this figure? To illustrate this, let's make a more final/professional version of this figure and increase the bins and segments to more meaningful values:

In [ ]:
atmospy.pollutionroseplot(
    data=met,
    ws="ws",
    wd="wd",
    pollutant="pm25",
    calm=0.1,
    bins=[0, 8, 15, 25, 35, 50, 100],
    segments=32,
    suffix="$µgm^{-3}$",
    title="$PM_{2.5}$ by Direction at an Unknown Location",
    cmap="viridis"
);

Alright, now what do we see? It seems to be that at this location, most of the PM2.5 is coming from the south-west and south-east. The longer the bar, the more data records that were grouped with that wind direction. The color of the bar indicates the intensity of the pollutant where the darker colors are higher values in this case. There doesn't seem to be any individual direction that stands out in terms of frequency of high PM2.5 values, which in and of itself is an insight.

The data above encompasses something like 7 months of data, so it's quite possible there is some seasonal or time of day influence that is hidden due to the averaging time shown above. We can begin to explore some of those questions in the section below.

## Advanced Features

As we've alluded to above, there is quite a bit we can do with the above 4 figures once we begin to subset or divide up our data based on engineered features. It can be incredibly useful to draw the same figure on different subsets of your data, which we refer to as "Faceting". We're going to walk through a couple of examples to illustrate (a) how to do this and (b) how powerful or insightful this can be.

I appologize in advance that the datasets we have available may not be the most beautiful and/or insightful. The process and idea still holds up, though!


**Exploring the influence of traffic on exposure**

Imagine you have a sensor along a major roadway and wanted to explore the influence of cars on exposure at this location. One very simple way to visualize this would be to look at the diel trend in CO, NOx, or Black Carbon, but do so separately for weekdays vs weekends when you expect traffic to be greatly reduced.

Let's illustrate this using the Black Carbon (`us-bc`) dataset we have available to us.

First, let's load the dataset and then choose a random location that we can use to illustrate our point:

In [ ]:
bc = atmospy.load_dataset("us-bc")

# select just one random location for now
bc_single_site = bc[bc["Local Site Name"] == bc["Local Site Name"].unique()[2]]

**Feature Engineering**

Next, we want to add a column that identifies the record as taking place on a weekday vs weekend. This is quite easy to do since we already have a column with the timestamp.

In [ ]:
# create a column that sets a bool if the date is a weekend
bc_single_site.loc[:, "Is Weekend"] = (
    bc_single_site["Timestamp Local"].dt.day_name().isin(["Saturday", "Sunday"])
)

bc_single_site.head()

Next, we want to convert our dataset to be long-form rather than wide-form, which we can do by using the `melt` function in `pandas` that we described above:

In [ ]:
# convert to long-form for faceting
bc_long_form = bc_single_site.melt(
    id_vars=["Timestamp Local", "Is Weekend"], value_vars=["Sample Measurement"]
)

bc_long_form.head()

Now that we have our engineered "Is Weekend" column, we can go ahead and set up a `FacetGrid` using `seaborn` and then plot the diel trend on top using the `dielplot` function from `atmospy`:

In [ ]:
g = sns.FacetGrid(
    data=bc_long_form,
    col="Is Weekend",
    # let's adjust the aspect ratio for funsies
    aspect=1.25,
)
g.map_dataframe(
    atmospy.dielplot, 
    x="Timestamp Local", 
    y="value", 
    plot_kws=dict(lw=4)
)

g.set(ylabel="Black Carbon");

While this may not be the greatest example, you can see a fairly stark difference between weekday and weekend Black Carbon trends at this location. As was printed out above, this site is in Hartford, CT and likely is seeing influence from the nearby roadway. You can see that:

  * The diel trend varies quite a bit day-to-day as shown by the large IQR
  * Weekdays see higher BC levels than weekends at this location
  * On weekdays, there is a pronounced morning rush-hour peak around 7 AM local time

Using faceting, we can quickly and clearly show the distinct difference between weekday and weekend exposure for Black Carbon at this location.

**Exploring Seasonal Differences in PM2.5 Source**

Depending on your location, it is quite possible that the source of whatever pollutant you're interested in changes seasonally. There are a number of ways in which you could visualize this, but for the sake of showcasing what you can do with `atmospy`, let's do so by plotting the pollution rose by month.

The dataset we have available (`air-sensors-met`) contains data for April through November. Like the previous figure, we're going to set up a `seaborn` `FacetGrid` to plot on. However, we first need to engineer a new column that contains the facet column (here, the month):

In [ ]:
# load the example dataset
met = atmospy.load_dataset("air-sensors-met")

# add a column that extracts the month from the timestamp_local column
met.loc[:, "Month"] = met["timestamp_local"].dt.month_name()

# print the first 5 records
met.head()

Then, we need to convert our data to be long-form rather than wide-form, as we did above:

In [ ]:
met_long_form = met.melt(
    id_vars=["timestamp_local", "Month", "ws", "wd"], 
    value_vars=["pm25"]
)

# print the first 5 records
met_long_form.head()

Last, we are going to set up a `FacetGrid` and direct it to use the `Month` column to facet on:

In [ ]:
# set up the FacetGrid
g = sns.FacetGrid(
    data=met_long_form,
    col="Month",
    col_wrap=3,
    subplot_kws={"projection": "polar"},
    despine=False,
)

# map the dataframe using the pollutionroseplot function
g.map_dataframe(
    atmospy.pollutionroseplot,
    ws="ws",
    wd="wd",
    pollutant="value",
    faceted=True,
    segments=16,
    bins=[0, 8, 15, 25, 35, 50, 100],
    calm=0.1,
    suffix="$µgm^{-3}$",
)

# add the legend and place it where it looks nice
g.add_legend(
    title="$PM_{2.5}$", bbox_to_anchor=(0.535, 0.2), handlelength=1, handleheight=1
);

# Reporting Issues or Feature Requests

If you decide to use `atmospy` and notice an issue, please file a bug report on the GitHub repository (show example). If you'd like to see a new feature added, please also report to the same GitHub repository. I can't promise anything, but I'll do my best.

# Wrapping Up

That's it for the walk-through. What can you make? Feel free to use your own data or one of the example datasets.